# M3.4 - Creating a Basin-Scale Water Budget

*Part of:* [**Open Science for Water Resources**](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources)

In [ ]:
import glob
import datetime
import earthaccess
import numpy as np
import h5py
import xarray as xr
import rasterio as rio
from matplotlib import pyplot

auth = earthaccess.login()

We wanted to compute a water budget for the Yellowstone River basin. So far, we've computed the inputs of the equation by calculating the monthly precipitation (P) rate.

$$
P = E + R + \Delta S
$$

Now, we need to compute the outputs side: the loss of water due to evapotranspiration (E) and runoff (R).

--- 

## Estimating evapotranspiration (ET) for our basin

**We'll use [NASA's MODIS MOD16 terrestrial evapotranspiration (ET) product](https://dx.doi.org/10.5067/MODIS/MOD16A3GF.061) to estimate how much water has been evaporated or transpired.**

The MODIS MOD16 product provides global estimates of ET. Because of its relatively high spatial resolution (500 m), MOD16 data are not provided as a single, global image. Instead, the data are provided as spatial tiles. [You can see a representation of the MODIS spatial grid system here.](https://modis-land.gsfc.nasa.gov/MODLAND_grid.html) Note that each tile is known by horizontal (h) and vertical (v) coordinates.

So, we'll need to first figure out which tile contains our study area.

**Let's open the basin Shapefile again so that we can get the [rectangular bounds (link)](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.bounds.html) of our basin.**

In [ ]:
import geopandas

basin = geopandas.read_file('/home/arthur.endsley/Workspace/NTSG/projects/Y2024_TOPS_Training/data/YellowstoneRiver_drainage_WSG84.shp')

In [ ]:
# Provided in (min_X, min_Y, max_X, max_Y) order
basin.bounds.to_numpy().ravel()

In [ ]:
bbox = basin.bounds.to_numpy().ravel()
bbox

Now, we're ready to download the tiles that intersect our study area.

- The `bounding_box` argument to `earthaccess.search_data()` allows us to provide a rectangular bounding box to use as a spatial filter. The argument to `bounding_box` must be a Python `tuple`.
- We're downloading a specific version of the MOD16 product, `MOD16A3GF`, where `GF` stands for "gap-filled," an annually reprocessed product that provides the best accuracy for annual ET totals.
- MOD16A3GF data come in annual time steps, so we want to filter our search to all data in the years 2014 through 2023.

In [ ]:
results = earthaccess.search_data(
    short_name = 'MOD16A3GF',
    temporal = ('2014-01-01', '2023-12-31'),
    bounding_box = tuple(bbox))

There are actually two tiles that intersect our study area. For simplicity, we'll ignore the very small part of our study area is located in a separate tile from the majority of our basin. Let's filter the results to just the MODIS tile we're interested in, based on its horizontal (h) and vertical (v) coordinates.

In [ ]:
results_clean = list(filter(lambda granule: 'h10v04' in granule['meta']['native-id'], results))

Then, let's download only the tiles we're interested in.

#### &#x1F3AF; Best Practice

Note that we've created a specific place for our MODIS MOD16 data: `data/MOD16A3`.

In [ ]:
earthaccess.download(results_clean, 'data/MOD16A3')

### Installing a new Python package

NASA data products are sometimes provided in scientific data formats that can be difficult to use. MOD16 is provided in the HDF4-EOS format, a special format for NASA spatial datasets. We can't open these data files using `xarray`, so we'll need to install a special library to do so.

**The `py4eos` library makes it easier to open HDF4-EOS data files.** To install it, make sure you have activate the virtual environment for our project, then use `pip`:

```sh
pip install py4eos
```

Let's open just one of the MOD16 data granules to see how `py4eos` works.

In [ ]:
import py4eos

hdf = py4eos.read_hdf4eos('data/MOD16A3/MOD16A3GF.A2014001.h10v04.061.2022077145846.hdf')
hdf

We can retrieve a dataset from an HDF4-EOS file using the `get()` method.

In [ ]:
et = hdf.get('ET_500m')
et

As with many scientific datasets, the data values are scaled so as to reduce the file size. That's the case for MOD16 data, too. The units of MOD16A3 data should be millimeters per year [mm year<sup>-1</sup>], equivalent to [kg m<sup>-2</sup> year<sup>-1</sup>], but the values above are much too large.

We can tell `py4eos` to apply the **scale** and **offset** to the data required to obtain the true, geophysical values.

In [ ]:
et = hdf.get('ET_500m', scale_and_offset = True)
et

And, instead of a NumPy array, we can ask `py4eos` to return a `rasterio` Dataset.

In [ ]:
ds_et = hdf.to_rasterio('ET_500m', filename = '', driver = 'MEM', scale_and_offset = True)
ds_et

### Compiling an ET time series for our basin

We wanted to get our ET data as a `rasterio` Dataset because we'd like to clip the ET data to the bounds of our basin.

Let's first choose an appropriate **coordinate reference system (CRS)** for our analysis. The MODIS MOD16 ET data are represented by a Sinusoidal projection, which is difficult to visualize. Let's choose [the Albers Equal-Area CRS (link)](https://epsg.io/5070) for our analysis.

In [ ]:
# Project our basin data to the new CRS
basin_albers = basin.to_crs(epsg = 5070)
basin_albers.geometry[0]

**We want to calculate the total ET of our basin for each year, so we'll need to work with all the yearly ET files we've downloaded.**

In [ ]:
# Get a list of the *.hdf files we downloaded
file_list = glob.glob('data/MOD16A3/*.hdf')
file_list

We'll need to project each MOD16 ET image to the new CRS before clipping it to our basin. Because `rasterio` can't work with HDF4 or HDF4-EOS files, we'll need to convert each MOD16 ET image to a GeoTIFF file, first. **Let's put the processed GeoTIFF files in a new folder, named `processed`.**

In [ ]:
gtiff_file_list = []

for filename in file_list:
    # Extract the year from the filename
    year = filename.split('.')[1].replace('A', '').replace('001', '')
    
    # Read in the MODIS MOD16 data
    hdf = py4eos.read_hdf4eos(filename)
    
    # Create a rasterio Dataset for this raster; also write out a GeoTIFF file
    output_filename = f'processed/MOD16A3GF_{year}_ET_500m.tiff'
    ds_et = hdf.to_rasterio(
        'ET_500m', filename = output_filename, scale_and_offset = True)
    ds_et.close()
    
    # Save the output GeoTIFF filename
    gtiff_file_list.append(output_filename)

#### &#x1F3AF; Best Practice

Note that we stored the output GeoTIFF files in a new folder, `processed`. We wouldn't want to confuse these files with our raw data!

In [ ]:
gtiff_file_list.sort()
gtiff_file_list

### Clipping a spatial dataset to a smaller study area

Now, we're ready to clip the MOD16 ET data to our basin. We can clip these datasets using `rasterio`, but we'd ultimately like to work with the data as an `xarray` Dataset. **Let's use `rioxarray`, again, so we can combine the strengths of the `rasterio` and `xarray` libraries.**

In [ ]:
import rioxarray

ds_et = rioxarray.open_rasterio(gtiff_file_list[0])

# Project the MOD16 data to match the CRS of our basin
ds_et_albers = ds_et.rio.reproject(basin_albers.crs)

# Clip the MOD16 data to the boundary of our basin; plot the result
ds_et_albers.rio.clip(basin_albers.geometry.values).plot()

What's nice about this plot is that we can actually see the Yellowstone River where it exits the basin, at the top-right of the image, near Sydney, Montana.

**Now, we're ready to do this for each our files. We'll go ahead and calculate the basin-wide ET in the same step, for each file.**

In [ ]:
# Create an empty list to store the basin-scale ET
et_series = []

for filename in gtiff_file_list:
    ds_et = rioxarray.open_rasterio(filename)
    
    # Project the MOD16 data to match the CRS of our basin
    ds_et_albers = ds_et.rio.reproject(basin_albers.crs)
    
    # Clip the MOD16 data to the boundary of our basin
    ds_et_basin = ds_et_albers.rio.clip(basin_albers.geometry.values)
    
    # Again, we take the mean because it is equivalent to spatially
    #    distributing the columns of water in each pixel
    et_series.append(float(ds_et_basin.mean().values))

et_series = np.array(et_series)
et_series

---

## Obtaining basin-scale runoff data

We now have one more piece of our basin-scale water budget. We're just missing runoff. This can be difficult to obtain in many areas; it's generally only available for *closed basins* where the output stream is gauged. This is the case for the Yellowstone River but also for thousands of basins world-wide. We obtained the gauge data from [HYSETS, which you can read more about here (link).](https://www.nature.com/articles/s41597-020-00583-2)

The HYSETS data are difficult to work with, so we've already done the work of subsetting the data to the Yellowstone River. HYSETS provides data on a number of things about a basin, but we're particularly interested in the **discharge** rate and the **drainage area.**

In [ ]:
import xarray as xr

ds = xr.open_dataset('/home/arthur/Workspace/NTSG/projects/Y2024_TOPS_Training/data/HYSETS-2023_watershed_YellowstoneRiver.nc')
ds

As we're currently just looking at a 10-year period from 2014 through 2023, let's slice the data to that time range.

In [ ]:
ds_10years = ds.sel(time = slice('2014-01-01', '2023-12-31'))
ds_10years['discharge'].plot()

Our discharge data appear to be in units of cubic meters per second, or [m<sup>3</sup> sec<sup>-1</sup>]. However, our precipitation and ET data are in millimeters per unit time. We'll need to do a conversion:

- First, because the HYSETS data are actually in daily time steps, we'll convert the discharge rates to [m<sup>3</sup> day<sup>-1</sup>].
- Then, we'll sum up the daily discharge rates to get the total discharge for a year.

In [ ]:
# Compute total daily discharge based on the mean rate
ds_10years['discharge_total'] = ds_10years.discharge * 60 * 60 * 24

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

# Compute annual discharge
ds_10years['discharge_total'].resample(time = 'YS').sum().to_series().plot.bar()

`xarray` doesn't know how to label the vertical axis for the plot, because we just created the new `'discharge_total'` dataset and didn't provide any metadata. But the units are now [m<sup>3</sup> year<sup>-1</sup>].

### Calculating basin area

The last step is to determine the area of our basin. We'll use the area to convert discharge rates, currently a volume per unit time, to a vertical height of water (millimeters) per unit time.

There are two ways we could calculate the area of our basin. First, HYSETS reports the basin area as the **drainage area.**

Second, because we have our basin mapped as polygon, we could simply calculate the area of that polygon. We want to make sure that we measure area in meters, not degrees. The CRS of our new basin Shapefile, below, indicates that the X and Y axes are measured in meters, so that's a good sign.

In [ ]:
basin_albers.crs

We can convert from square meters to square kilometers easily.

In [ ]:
basin_area = basin_albers.area.values

# Convert from [square meters] to [square kilometers]
basin_area / 1e6

How does this compare to the drainage area reported by HYSETS? It's pretty close!

In [ ]:
ds_10years.drainage_area.values

### Computing basin-scale runoff

Whichever estimate of the basin area we use, we can convert discharge volume to discharge height by dividing by the basin area. The result is an estimate of how much water, in millimeters, drained through the Yellowstone River each year if the entire basin were flooded to the same height.

In [ ]:
# Get the [m year-1] that this basin drained through Yellowstone River,
#  then convert from [m year-1] to [mm year-1]
runoff_meters_per_yr = ds_10years['discharge_total'].resample(time = 'YS').sum() / basin_area
runoff_mm_per_yr = runoff_meters_per_yr * 1000

runoff_series = runoff_mm_per_yr.values
runoff_series

---

## Putting it all together

We're now ready to complete our water budget. In this exercise, we're interested in how water inputs (precipitation) and water outputs (ET and runoff) balance, with the result being the estimated change in groundwater storage, $\Delta S$. We can re-arrange the terms of our water budget equation to obtain a formula for $\Delta S$.

$$
P = E + R + \Delta S \quad\rightarrow\quad \Delta S = P - (E + R)
$$

Let's open the precipitation data we created earlier.

In [ ]:
ds_precip = xr.open_dataset('processed/IMERG-Final_precip_monthly_2014-2023.nc')

precip_series = ds_precip.precip_monthly.mean(['lon','lat']).groupby('time.year').sum().values
precip_series

In [ ]:
et_series

In [ ]:
runoff_series

Before we compute $\Delta S$, let's compare these three water fluxes together over time. As we would expect, they are moderately correlated: lower precipitation levels would certainly imply lower levels of ET and runoff. We can see that 2020 and 2021 were relatively dry years for this basin.

In [ ]:
years = np.arange(2014, 2024)
pyplot.plot(years, precip_series, 'b', label = 'Precipitation')
pyplot.plot(years, et_series, 'g', label = 'ET')
pyplot.plot(years, runoff_series, 'r', label = 'Runoff')
pyplot.ylabel('Water Flux (mm per year)')
pyplot.legend()
pyplot.show()

In [ ]:
delta_storage = precip_series - (et_series + runoff_mm_per_yr.values)
delta_storage

**The final result gives us an estimate of how groundwater storage may have been affected by the balance of water inputs and outputs over this 10-year time period.**

In [ ]:
pyplot.bar(years, height = delta_storage)